# 1. Import the required libraries

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import geemap
import json
import os
import requests
from geemap import geojson_to_ee, ee_to_geojson
from ipyleaflet import GeoJSON, Marker, MarkerCluster

# 2. Load the data

## Loading the forest inventory table
There are two regions of interest in the Democratic Republic of Congo which we have selected, namely `Equateur` and `Orientale`.

In [2]:
gdf = gpd.read_file('./shapes/shapes.shp')
gdf

,Country,Region,to_crs,PID,geometry
0,Congo,Likouala,epsg:4057,10000,"POLYGON ((17.60632 3.64125, 17.65328 3.63904, ..."
1,Congo,Sangha,epsg:4057,10001,"POLYGON ((16.46330 2.70887, 16.47159 2.70536, ..."
2,Democratic Republic of the Congo,Équateur,epsg:4057,10002,"POLYGON ((19.60209 5.13152, 19.61267 5.12790, ..."
3,Democratic Republic of the Congo,Orientale,epsg:4057,10003,"POLYGON ((25.55505 5.37646, 25.56033 5.37533, ..."
4,USA,Sierra,epsg:26943,10004,"MULTIPOLYGON (((-119.77867 37.46773, -119.7786..."
5,USA,Los Padres,epsg:6423,10005,"MULTIPOLYGON (((-120.74838 35.44383, -120.7483..."


## Loading the plot information data (collected from the projects)
There are two regions of interest in the Democratic Republic of Congo which we have selected, namely `Equateur` and `Orientale`.

In [3]:
df = pd.read_csv('./data/Plot information.csv')
df

,Plot ID,Plot Name,Project Start,Project End,Project Owner,Project Type,Budget,Partners,ER-PA Signatory,Certified by,Cerified by,Lead Verifier
0,10000,Likouala,2019,2023,World Bank Forest Carbon Partnership Facility ...,"REDD+ Climate, Community and Biodiversity",USD 54.6m,"Forest Investment Program (FIP), the Central A...","Government of the Republic of Congo, through t...",Gold Standard,SCS Global Services,M. Calixte Nganongo
1,10001,Dzangha-Sangha,2019,2023,World Bank Forest Carbon Partnership Facility ...,Improved Forest Management,USD 38m,"Forest Investment Program (FIP), the Central A...","Government of the Republic of Congo, through t...",VM0004v1.0,SCS Global Services,Georges Claver Boundzanga
2,10002,Equateur,2016,2023,Govt. of the Democratic Republic of the Congo,Avoided conservation,USD 1.15m,World Bank Forest Carbon Partnership Facility ...,"Ministry of Environment, Conservation of Natur...",Verra,SCS Global Services,Vincent Kasulu Seya Makonga
3,10003,Orientale,2019,2026,Govt. of the Democratic Republic of the Congo,REDD+ Avoided Planned Deforestation,USD1.2m,World Bank Forest Carbon Partnership Facility ...,"Ministry of Environment, Conservation of Natur...","CCB (climate, community and biodiversity allia...",SCS Global Services,M. Leon Kanu Mbizi
4,10004,Sierra,2015,2025,Calfornia Climate Investments (CCI),Improved Forest Management,USD 2m,"Sonoma County, State of California",State of California,"Climate Action Reserve Project Protocol v3.1,",SCS Global Services,Francis Eaton
5,10005,Los Padres,2014,2024,Los Padres LLC,REDD+ Avoided Planned Deforestation,USD 3.4m,"Sonoma County, State of California",State of California,American Carbon Registry,SCS Global Services,Zane Haxtema


# 2. Merge the data

## Loading the plot information data (collected from the projects)
There are two regions of interest in the Democratic Republic of Congo which we have selected, namely `Equateur` and `Orientale`.

In [4]:
df.rename(columns={"Plot ID":"PID"}, inplace=True)
gdf = pd.concat([gdf, df], axis=1)
gdf.drop(columns=gdf.columns[3],inplace=True)
gdf['PID'] = ['10000', '10001', '10002', '10003', '10004', '10005']
gdf.set_index("PID", inplace=True)
gdf

,Country,Region,to_crs,geometry,Plot Name,Project Start,Project End,Project Owner,Project Type,Budget,Partners,ER-PA Signatory,Certified by,Cerified by,Lead Verifier
PID,,,,,,,,,,,,,,,
10000,Congo,Likouala,epsg:4057,"POLYGON ((17.60632 3.64125, 17.65328 3.63904, ...",Likouala,2019,2023,World Bank Forest Carbon Partnership Facility ...,"REDD+ Climate, Community and Biodiversity",USD 54.6m,"Forest Investment Program (FIP), the Central A...","Government of the Republic of Congo, through t...",Gold Standard,SCS Global Services,M. Calixte Nganongo
10001,Congo,Sangha,epsg:4057,"POLYGON ((16.46330 2.70887, 16.47159 2.70536, ...",Dzangha-Sangha,2019,2023,World Bank Forest Carbon Partnership Facility ...,Improved Forest Management,USD 38m,"Forest Investment Program (FIP), the Central A...","Government of the Republic of Congo, through t...",VM0004v1.0,SCS Global Services,Georges Claver Boundzanga
10002,Democratic Republic of the Congo,Équateur,epsg:4057,"POLYGON ((19.60209 5.13152, 19.61267 5.12790, ...",Equateur,2016,2023,Govt. of the Democratic Republic of the Congo,Avoided conservation,USD 1.15m,World Bank Forest Carbon Partnership Facility ...,"Ministry of Environment, Conservation of Natur...",Verra,SCS Global Services,Vincent Kasulu Seya Makonga
10003,Democratic Republic of the Congo,Orientale,epsg:4057,"POLYGON ((25.55505 5.37646, 25.56033 5.37533, ...",Orientale,2019,2026,Govt. of the Democratic Republic of the Congo,REDD+ Avoided Planned Deforestation,USD1.2m,World Bank Forest Carbon Partnership Facility ...,"Ministry of Environment, Conservation of Natur...","CCB (climate, community and biodiversity allia...",SCS Global Services,M. Leon Kanu Mbizi
10004,USA,Sierra,epsg:26943,"MULTIPOLYGON (((-119.77867 37.46773, -119.7786...",Sierra,2015,2025,Calfornia Climate Investments (CCI),Improved Forest Management,USD 2m,"Sonoma County, State of California",State of California,"Climate Action Reserve Project Protocol v3.1,",SCS Global Services,Francis Eaton
10005,USA,Los Padres,epsg:6423,"MULTIPOLYGON (((-120.74838 35.44383, -120.7483...",Los Padres,2014,2024,Los Padres LLC,REDD+ Avoided Planned Deforestation,USD 3.4m,"Sonoma County, State of California",State of California,American Carbon Registry,SCS Global Services,Zane Haxtema


# 3. Save the data

## Loading the plot information data (collected from the projects)
There are two regions of interest in the Democratic Republic of Congo which we have selected, namely `Equateur` and `Orientale`.

In [5]:
gdf.to_file("./shapes/plot_info.shp")
gdf.to_file("./shapes/geojson/plot_info.geojson", driver='GeoJSON')

/tmp/ipykernel_123970/1956890688.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  gdf.to_file("./shapes/plot_info.shp")


## Loading the plot information data (collected from the projects)
There are two regions of interest in the Democratic Republic of Congo which we have selected, namely `Equateur` and `Orientale`.

In [6]:
#computing polygon centroids
gdf['centroid'] = gdf.centroid
gdf

/tmp/ipykernel_123970/713158588.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['centroid'] = gdf.centroid


,Country,Region,to_crs,geometry,Plot Name,Project Start,Project End,Project Owner,Project Type,Budget,Partners,ER-PA Signatory,Certified by,Cerified by,Lead Verifier,centroid
PID,,,,,,,,,,,,,,,,
10000,Congo,Likouala,epsg:4057,"POLYGON ((17.60632 3.64125, 17.65328 3.63904, ...",Likouala,2019,2023,World Bank Forest Carbon Partnership Facility ...,"REDD+ Climate, Community and Biodiversity",USD 54.6m,"Forest Investment Program (FIP), the Central A...","Government of the Republic of Congo, through t...",Gold Standard,SCS Global Services,M. Calixte Nganongo,POINT (17.44950 2.08502)
10001,Congo,Sangha,epsg:4057,"POLYGON ((16.46330 2.70887, 16.47159 2.70536, ...",Dzangha-Sangha,2019,2023,World Bank Forest Carbon Partnership Facility ...,Improved Forest Management,USD 38m,"Forest Investment Program (FIP), the Central A...","Government of the Republic of Congo, through t...",VM0004v1.0,SCS Global Services,Georges Claver Boundzanga,POINT (15.36175 1.37380)
10002,Democratic Republic of the Congo,Équateur,epsg:4057,"POLYGON ((19.60209 5.13152, 19.61267 5.12790, ...",Equateur,2016,2023,Govt. of the Democratic Republic of the Congo,Avoided conservation,USD 1.15m,World Bank Forest Carbon Partnership Facility ...,"Ministry of Environment, Conservation of Natur...",Verra,SCS Global Services,Vincent Kasulu Seya Makonga,POINT (20.60453 1.04351)
10003,Democratic Republic of the Congo,Orientale,epsg:4057,"POLYGON ((25.55505 5.37646, 25.56033 5.37533, ...",Orientale,2019,2026,Govt. of the Democratic Republic of the Congo,REDD+ Avoided Planned Deforestation,USD1.2m,World Bank Forest Carbon Partnership Facility ...,"Ministry of Environment, Conservation of Natur...","CCB (climate, community and biodiversity allia...",SCS Global Services,M. Leon Kanu Mbizi,POINT (26.36593 2.09265)
10004,USA,Sierra,epsg:26943,"MULTIPOLYGON (((-119.77867 37.46773, -119.7786...",Sierra,2015,2025,Calfornia Climate Investments (CCI),Improved Forest Management,USD 2m,"Sonoma County, State of California",State of California,"Climate Action Reserve Project Protocol v3.1,",SCS Global Services,Francis Eaton,POINT (-119.20355 37.26252)
10005,USA,Los Padres,epsg:6423,"MULTIPOLYGON (((-120.74838 35.44383, -120.7483...",Los Padres,2014,2024,Los Padres LLC,REDD+ Avoided Planned Deforestation,USD 3.4m,"Sonoma County, State of California",State of California,American Carbon Registry,SCS Global Services,Zane Haxtema,POINT (-119.91449 35.01119)


In [7]:
gdf.drop(columns="geometry", inplace=True)
gdf.rename(columns={"centroid": "geometry"}, inplace=True)
gdf
gdf.to_file("./shapes/geojson/plot_info_centroids.geojson", driver='GeoJSON')

# 3. Visualize the vector data to check accuracy

There are two regions of interest in the Democratic Republic of Congo which we have selected, namely `Equateur` and `Orientale`.

In [10]:
#Point cluster plot (centroids)
Map = geemap.Map()
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

---
Copyright © 2022 Carbon Capital Advisors. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.